In [5]:
import pandas as pd
from sqlalchemy import create_engine
from binance.client import Client

client = Client()
engine = create_engine('sqlite:///crypto_1m.db')

BinanceAPIException: APIError(code=0): Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error.

In [4]:
def getminutedata(symbol, start):
    frame = pd.DataFrame(client.get_historical_klines(symbol, '1m', start))
    frame = frame[[0,1,2,3,4,5]]
    frame.columns = ['Date', 'Open', 'High', 'Low', 'Close']
    frame.Date = pd.to_datetime(frame.Date, unit='ms')
    frame.set_index(['Date'],inplace=True)
    frame = frame.astype(float)
    return frame

In [7]:
symbols = pd.read_csv('symbols.csv')
for symbol in symbols.name:
    df = getminutedata(symbol,'80 days ago UTC')
    df.to_sql(symbol, engine)

NameError: name 'client' is not defined

In [11]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import timedelta

In [ ]:
class SmaCross(Strategy):
    n1 = 50
    n2 = 100
    
    def init(self):
        close = self.data.Close
        self.sma1 = self.I({SMA, close, self.n1})
        self.sma2 = self.I({SMA, close, self.n2})

    def next(self):
        price = self.data.Close
        if crossover(self.sma1, self.sma2):
            sl = price - price*0.03
            tp = price + price*0.04
            self.buy(sl = sl, tp = tp)
    

In [13]:
def resampleOHLC(df, interval):
    df.resample(interval).agg({
        'Open':'first',
        'High':'max',
        'Low':'min',
        'Close':'last'})
    return df


In [14]:
symbols = pd.read_sql("""SELECT name FROM sqlite_schema WHERE type='table'""", engine)

NameError: name 'engine' is not defined

In [ ]:
def BT(interval): 
    
    returns = []

    for symbol in symbols.name: 
        qry = f"""SELECT * FROM '{symbol}' 
            WHERE Date < '{pd.to_datetime('today') - timedelta(days=30)}'"""
        df = pd.read_sql(qry, engine).set_index('Date')
        df.index = pd.to_datetime(df.index)
        df = resampleOHLC(df, interval)
        bt = Backtest(df, SmaCross, cash=1000, commission=0.0015, exclusive_orders=True)
        output = bt.run()
        returns.append(output['Return [%]'])

    frame = pd.DataFrame(returns, index=symbol.name, columns=['ret'])
    top_5 = frame.nlargest(5, 'ret')
    return top_5, interval


In [ ]:
top_5, interval = BT('15min')

In [15]:
def validate(top_5, interval):

    returns = []

    for symbol in top_5.index:
        qry = f"""SELECT * FROM '{symbol}' 
            WHERE Date > '{pd.to_datetime('today') - timedelta(days=30)}'"""
        df = pd.read_sql(qry, engine).set_index('Date')
        df.index = pd.to_datetime(df.index)
        df = resampleOHLC(df, interval)
        bt = Backtest(df, SmaCross, cash=1000, commission=0.0015, exclusive_orders=True)
        output = bt.run()
        returns.append(output['Return [%]'])

    outcome = pd.DataFrame(returns, index=top_5.index, columns=['ret'])
    return outcome

In [ ]:
validate(top_5, interval)

In [ ]:
def run(interval):
    bt_result, interval = BT(interval)
    valid = validate(bt_result,interval)
    return valid.ret.mean()


In [ ]:
run('15min')

In [ ]:
int_scale = 'min'

for i in range(15, 75, 15):
    print('return for '+str(i) + int_scale)
    print(run(str(i)+int_scale))

In [2]:
import ccxt
kraken = ccxt.kraken() 
ohlcv = kraken.fetch_ohlcv('BTC/USDT', timeframe='1h')
print(ohlcv[:2])

[[1746536400000, 93943.9, 93979.0, 93475.2, 93629.8, 2.48592971], [1746540000000, 93679.2, 94600.0, 93679.1, 94123.1, 13.6855674]]
